<a href="https://colab.research.google.com/github/fikarrizkip/Marketing-and-Customer-Analytics/blob/main/A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
Perusahaan telah mengembangkan halaman web baru dengan tujuan meningkatkan jumlah pengguna berbayar untuk produk mereka. Untuk mengukur efektivitas halaman baru tersebut, mereka melakukan uji A/B.

Uji ini melibatkan pembagian acak pengguna ke dalam dua kelompok berukuran sama: kelompok kontrol (A) dan kelompok perlakuan (B). Kelompok kontrol tetap mengakses halaman web lama, sementara kelompok perlakuan berinteraksi dengan halaman web yang baru dikembangkan.

Setelah periode pengujian tertentu, data dikumpulkan dari kedua kelompok. Selanjutnya, pengujian hipotesis diterapkan untuk menentukan apakah perbedaan antara kedua kelompok tersebut secara statistik signifikan atau tidak. Dengan cara ini, perusahaan dapat menarik kesimpulan yang valid tentang dampak halaman web baru terhadap perilaku pengguna dan pengambilan keputusan pembelian.

#Langkah-langkah A/B Testing

**Membuat Hipotesis**

H0: Tidak ada perbedaan yang signifikan secara statistik antara halaman lama dan halaman baru

H1: Ada perbedaan yang signifikan secara statistik antara halaman lama dan halaman baru
<br></br>

**Memeriksa Asumsi**

H0: Asumsi distribusi normal terpenuhi

H1: Asumsi distribusi normal tidak terpenuhi

Jika nilai p-value kurang dari 0,05, itu dianggap signifikan dan tes non-parametrik (uji mannwhitneyu) akan digunakan. Lain tes parametrik (uji-t)
<br></br>
**Memeriksa Homogenitas**

H0: Varians homogen

H1: Varians tidak homogen

# Exploratory Data Analysis

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import shapiro, levene, mannwhitneyu
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [2]:
#load data
from google.colab import drive
drive.mount('/content/drive')
ab = pd.read_excel("/content/drive/MyDrive/dataset/ab_data_excel.xlsx")

Mounted at /content/drive


In [3]:
#overview data
ab.head(5)

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [4]:
#descriptive stats
ab.describe()

,user_id,converted
count,294480.000000,294480.000000
mean,787973.538896,0.119658
std,91210.917091,0.324562
min,630000.000000,0.000000
25%,709031.750000,0.000000
50%,787932.500000,0.000000
75%,866911.250000,0.000000
max,945999.000000,1.000000


In [5]:
#checking data type
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [6]:
#checking missing values
ab.isnull().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [7]:
#checking data duplicates
ab_duplicate = ab.duplicated(['user_id'])
print(ab_duplicate)

0         False
1         False
2         False
3         False
4         False
          ...  
294475    False
294476    False
294477    False
294478     True
294479    False
Length: 294480, dtype: bool


In [8]:
print(ab.shape)

(294480, 5)


In [9]:
ab = ab.drop_duplicates(subset='user_id', keep=False)
print(ab.shape)

(286690, 5)


In [10]:
#mengelompokkan kategori grup dan kontrol
ab[((ab['group'] == 'control') & (ab['landing_page'] == 'new_page')) | ((ab['group'] == 'treatment') & (ab['landing_page'] == 'old_page'))]

,user_id,timestamp,group,landing_page,converted


# A/B Testing

**Membuat Hipotesis**
<br>H0: Tidak ada perbedaan yang signifikan secara statistik antara halaman lama dan halaman baru</br>

H1: Terdapat perbedaan yang signifikan secara statistik antara halaman lama dan halaman baru

**Pengecekan Asumsi**
<br>Asumsi Normalitas</br>
H0: Asumsi distribusi normal terpenuhi
<br>H1: Asumsi distribusi normal tidak terpenuhi
Jika nilai p kurang dari 0,05, dianggap signifikan, dan uji non-parametrik (uji Mann-Whitney) akan digunakan. Jika tidak, uji parametrik (uji t)</br>

**Homogenitas Varians**

H0: Varian homogen
<br>H1: Varian tidak homogen</br>

**Kesimpulan**
<br>Hipotesis akan disimpulkan berdasarkan nilai p yang diperoleh dari uji parametrik/non-parametrik yang akan kita lakukan. Hasil ini akan menjawab pertanyaan: Apakah ada perbedaan signifikan antara halaman baru dan halaman lama?</br>

**Memeriksa Jenis Data**

In [11]:
test_stat, pvalue = shapiro(ab.loc[ab["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.3792334198951721


In [12]:
test_stat, pvalue = shapiro(ab.loc[ab["landing_page"] == "new_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.37685757875442505


In [13]:
# Menentukan apakah data berasal dari distribusi normal
alpha = 0.05
if pvalue < alpha:
    print("Data tidak berasal dari distribusi normal")
else:
    print("Data berasal dari distribusi normal")

Data tidak berasal dari distribusi normal


Karena p-value < 0.05, maka asumsi distribusi normal tidak terpenuhi dan akan menggunakan Uji Mann-Whitney untuk langkah selanjutnya.

In [14]:
test_stat, pvalue = levene(ab.loc[ab["landing_page"] == "new_page", "converted"],
                           ab.loc[ab["landing_page"] == "old_page", "converted"])
print("p-value:",pvalue)
print("test_stat:",test_stat)
alpha = 0.05
if pvalue < alpha:
    print("Varians tidak homogen")
else:
    print("Varians homogen")

p-value: 0.2322897281547632
test_stat: 1.4267917566652295
Varians homogen


**Uji Mann-Whitney**
<br>Hipotesis:</br>

**H0 (Hipotesis Nol):** Tidak terdapat perbedaan signifikan secara statistik antara halaman lama dan halaman baru.

**H1 (Hipotesis Alternatif):** Terdapat perbedaan signifikan secara statistik antara halaman lama dan halaman baru.


In [15]:
test_stat, pvalue = mannwhitneyu(ab.loc[ab["landing_page"] == "new_page", "converted"],
                                 ab.loc[ab["landing_page"] == "old_page", "converted"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 10259026653.0000, p-value = 0.2323


In [16]:
alpha = 0.05
if pvalue > alpha:
    print("Gagal menolak H0")
else:
    print("H0 ditolak")

Gagal menolak H0


#Conclusion
Berdasarkan hasil pengujian, **kita gagal menolak hipotesis nol (H0)**. Hal ini berarti **tidak ada perbedaan signifikan secara statistik** antara halaman lama dan halaman baru dalam hal perilaku pengguna dan keputusan pembelian.

Dengan kata lain, berdasarkan data yang diperoleh, **belum dapat disimpulkan** bahwa halaman web baru dapat **meningkatkan jumlah pengguna berbayar** secara signifikan. Oleh karena itu, **belum tentu** halaman web baru tersebut dapat meningkatkan profit perusahaan.  

#Rekomendasi
Langkah selanjutnya dapat melibatkan beberapa pertimbangan dan strategi berikut:
<br></br>

1. Analisis Lebih Lanjut:

Periksa hasil secara lebih mendalam, seperti melihat statistik deskriptif, mengeksplorasi pola data, dan menganalisis segmentasi pengguna. Ini dapat membantu dalam pemahaman lebih lanjut terkait dengan perilaku pengguna pada kedua halaman.
<br></br>
2. Feedback Pengguna:

Kumpulkan umpan balik dari pengguna terkait dengan halaman web baru. Mungkin ada aspek tertentu yang dapat diperbaiki atau dioptimalkan untuk meningkatkan pengalaman pengguna.
<br></br>
3. A/B Testing Iteratif:

Jika mungkin, pertimbangkan untuk melakukan iterasi lebih lanjut dari uji A/B dengan memodifikasi halaman web baru berdasarkan temuan dan umpan balik sebelumnya.
<br></br>
4. Pengembangan Fitur atau Konten:

Tinjau elemen-elemen tertentu pada halaman web baru yang mungkin mempengaruhi keputusan pembelian. Pertimbangkan pengembangan atau perubahan dalam hal fitur, desain, atau konten.
<br></br>
5. Pertimbangkan Faktor Eksternal:

Tinjau faktor-faktor eksternal yang mungkin mempengaruhi hasil, seperti perubahan tren pasar, musim, atau perubahan dalam perilaku pengguna secara keseluruhan.
<br></br>
6. Melibatkan Tim Pemasaran dan Desain:

Dapatkan wawasan dari tim pemasaran dan desain mengenai elemen-elemen apa yang mungkin berkontribusi pada keberhasilan atau kegagalan halaman web baru.
<br></br>
7. Analisis Biaya dan Manfaat:

Tinjau kembali analisis biaya dan manfaat dari perubahan halaman web. Pertimbangkan apakah investasi lebih lanjut dalam pengembangan halaman web baru masih sebanding dengan potensi keuntungan yang mungkin diperoleh.
<br></br>
8. Pemilihan Metode Analisis Alternatif:

Pertimbangkan metode analisis alternatif atau lebih canggih yang mungkin lebih sesuai dengan karakteristik data atau desain eksperimen tertentu.
<br></br>
9. Konsultasi Ahli Statistik:

Jika mungkin, konsultasikan hasil dan metode dengan ahli statistik untuk mendapatkan pandangan dan rekomendasi yang lebih mendalam.